In [ ]:
from easyquant import MongoIo
from easyquant.indicator.base import *
import math
import numpy as np
import pandas as pd

In [ ]:
m=MongoIo()

In [32]:
data=m.get_stock_day('600822',st_start='2019-01-01', st_end='2019-12-07')

In [33]:
C=data.close
H=data.high
L=data.low

In [34]:
M=144
N=13
# A1=(C-MA(C,M))/MA(C,M)*100
# N1=BARSLAST(CROSS(C,MA(C,M)))
# N2=BARSLAST(CROSS(MA(C,M),C))
# B1=IF(N1<N2,N1+1,0)
# C1=HHV(A1,B1)


In [43]:
N3 = BARSLAST(CROSS(C, MA(C, N)))
N4 = BARSLAST(CROSS(MA(C, N), C))
AA = IF(N3 < N4, N3 + 1, 0)
BB = (C - REF(C, AA[-1])) / REF(C, AA[-1]) * 100
MR = IFAND(AA == 1 , BB > AA,1,0)


In [42]:
BB.tail()

date
2019-12-02    0
2019-12-03    0
2019-12-04    0
2019-12-05    0
2019-12-06    0
dtype: int32

In [21]:
N.index - N

AttributeError: 'DatetimeIndex' object has no attribute 'len'

In [ ]:
N.index - N

In [23]:
B1[0]

0

In [ ]:
D1=(C-REF(C,B1[0]))/REF(C,B1)*100
N3=BARSLAST(CROSS(C,MA(C,N)))
N4=BARSLAST(CROSS(MA(C,N),C))
AA=IF(N3<N4,N3+1,0)
BB=(C-REF(C,AA))/REF(C,AA)*100,COLORRED
MR=(AA == 1 and BB>AA)


In [ ]:
COND2=#BB0=REF(QSX,1)<11 and CROSS(QSX,11) and C<ZX;
BB01=IFAND(REF(QSX,1)<11,CROSS(QSX,11),True, False)
BB0=IFAND(BB01,C<ZX,True, False)

In [ ]:
COND1=BB01
COND2=C<ZX

In [ ]:
COND2[COND2.index>=COND1.index[0]]

In [ ]:
type(COND1)

In [ ]:
COND2.index

In [ ]:
len(COND1)

In [ ]:
len(COND2)

In [ ]:
an[an.index>=BB01.index[0]]

In [ ]:
C<ZX

In [ ]:
AA1=(QSX<11)

In [ ]:
AA2=FILTER((QSX<=11),15)

In [ ]:
AA3= C<ZX

In [ ]:

BB0=REF(QSX,1)<11 and CROSS(QSX,11) and C<ZX;
BB1=REF(QSX,1)<11 and REF(QSX,1)>6 and CROSS(QSX,11);
BB2=REF(QSX,1)<6 and REF(QSX,1)>3 and CROSS(QSX,6);
BB3=REF(QSX,1)<3 and REF(QSX,1)>1 and CROSS(QSX,3);
BB4=REF(QSX,1)<1 and REF(QSX,1)>0 and CROSS(QSX,1);
BB5=REF(QSX,1)<0 and CROSS(QSX,0);
BB=BB1=1 OR BB2=1 OR BB3=1 OR BB4=1 OR BB5=1;
BUY:STICKLINE(BB=1 and C<ZX,11,52,1,0);
DD0=REF(QSX,1)>89 and CROSS(89,QSX) and C>ZX;
DD1=REF(QSX,1)>89 and REF(QSX,1)<94 and CROSS(89,QSX);
DD2=REF(QSX,1)>94 and REF(QSX,1)<97 and CROSS(94,QSX);
DD3=REF(QSX,1)>97 and REF(QSX,1)>99 and CROSS(97,QSX);
DD4=REF(QSX,1)>99 and REF(QSX,1)<100 and CROSS(99,QSX);
DD5=REF(QSX,1)>100 and CROSS(100,QSX);
DD=DD1=1 OR DD2=1 OR DD3=1 OR DD4=1 OR DD5=1;
SELL=STICKLINE(DD=1 and C>ZX,89,49,1,0);